In [2]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [7]:
data = pd.read_csv("train_titanic.csv")

In [8]:
df = data.copy()

## As for lot of pasengers age is not availble we fill it with means of all ages

In [22]:
mean_age = df['Age'].mean()

In [23]:
mean_age

29.700559701492537

In [24]:
values = {"Age": mean_age}

In [25]:
df.fillna(value = values, inplace = True)

## ----------------------------------
## Clean Data

In [29]:
df['Embarked'] = df['Embarked'].astype('category').cat.codes
df['Sex'] = df['Sex'].astype('category').cat.codes

X_dataset = df[df.columns.difference(['Survived', 'Name', 'Ticket', 'Age'])]
Y_dataset = df['Survived']

X =  X_dataset.values



Y = Y_dataset.values

## Spliting Data To Train and  Test 

In [34]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X , Y )